In [1]:
import numpy as np
import pandas as pd
from board import *

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# ToDo:
# Implement property upgrading

In [3]:
players = [Player(0.5), Player(0.5), Player(0.5), Player(0.5)]

In [4]:
for i in range(100000):
    curr_player = players[i % 4]
    curr_player.take_turn()

In [5]:
squares = np.array([square.name for square in board])
visits = np.array([square.visits for square in board])
visits = visits / sum(visits)
profits = []
for square in board:
    if issubclass(type(square), Property):
        profits.append(square.profit)
    else:
        profits.append(0)
profits = np.array(profits) / sum(profits)

In [11]:
data = pd.DataFrame({
    "Squares": squares, "Visits": visits, "Profits": profits
}).sort_values(by="Profits", ascending=False)

In [12]:
data

,Squares,Visits,Profits
39,Boardwalk,0.023416,0.096609
5,Reading Railroad,0.027006,0.055724
37,Park Place,0.019170,0.055310
34,Pennsylvania Avenue,0.023655,0.054614
32,North Carolina Avenue,0.025302,0.054253
25,B&O Railroad,0.025846,0.053328
15,Pennsylvania Railroad,0.025709,0.053044
24,Illinois Avenue,0.031343,0.051718
31,Pacific Avenue,0.023711,0.050832
26,Atlantic Avenue,0.026380,0.047865
